# ACCRE Project - Data Import and Cleaning

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
accre = pd.read_csv("../data/fullsample.csv")

In [3]:
accre

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
0,30616928,RUNNING,2021-07-31T22:15:00,Unknown,2048Mn,0,10:04:00,67-22:14:22,1,1,production,0:0
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05T11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05T19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07T20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10T16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0
...,...,...,...,...,...,...,...,...,...,...,...,...
7395880,25493434,COMPLETED,2020-10-31T23:39:00,2020-10-31T23:40:46,2000Mn,0.09M,2-00:00:00,00:01:46,1,1,sam,0:0
7395881,25493435,COMPLETED,2020-10-31T23:39:13,2020-10-31T23:40:38,2000Mn,187.92M,2-00:00:00,00:01:25,1,1,sam,0:0
7395882,25493476,COMPLETED,2020-10-31T23:46:29,2020-10-31T23:49:43,4096Mc,803.97M,12:00:00,00:03:14,1,1,production,0:0
7395883,25493515,COMPLETED,2020-10-31T23:49:44,2020-10-31T23:51:40,2000Mn,0.09M,2-00:00:00,00:01:56,1,1,sam,0:0


In [4]:
accre['BEGIN'] = pd.to_datetime(accre['BEGIN'], errors='coerce')
accre['END'] = pd.to_datetime(accre['END'], errors='coerce')

In [5]:
accre = accre.sort_values('END')

In [6]:
accre

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
6640577,24460577,COMPLETED,2020-10-01 00:05:55,2020-10-01 00:10:15,4096Mc,868.77M,12:00:00,00:04:20,1,1,production,0:0
6640635,24460647,COMPLETED,2020-10-01 00:10:38,2020-10-01 00:12:58,2000Mn,0.09M,2-00:00:00,00:02:20,1,1,sam,0:0
6640627,24460639,COMPLETED,2020-10-01 00:11:22,2020-10-01 00:13:31,1024Mc,221.10M,00:30:00,00:02:09,1,1,production,0:0
6640651,24460668,COMPLETED,2020-10-01 00:15:16,2020-10-01 00:17:23,2000Mn,0.09M,2-00:00:00,00:02:07,1,1,sam,0:0
6640654,24460671,COMPLETED,2020-10-01 00:17:08,2020-10-01 00:17:53,1024Mc,636.96M,00:30:00,00:00:45,1,1,production,0:0
...,...,...,...,...,...,...,...,...,...,...,...,...
927267,32922493,RUNNING,2021-10-07 09:13:25,NaT,186648Mn,0,2-00:00:00,11:15:57,1,32,nogpfs,0:0
927268,32922526,RUNNING,2021-10-07 09:18:25,NaT,62193Mn,0,2-00:00:00,11:10:57,1,12,nogpfs,0:0
927269,32922527,RUNNING,2021-10-07 09:25:25,NaT,46066Mn,0,2-00:00:00,11:03:57,1,12,nogpfs,0:0
927270,32922542,RUNNING,2021-10-07 09:36:25,NaT,186648Mn,0,2-00:00:00,10:52:57,1,32,nogpfs,0:0


In [16]:
accre[accre['END'].notnull()].rolling('1h', on = 'END')['JOBID'].count()

6640577    1.0
6640635    2.0
6640627    3.0
6640651    4.0
6640654    5.0
          ... 
927386     9.0
927387     9.0
927388     8.0
1491977    9.0
1491978    9.0
Name: JOBID, Length: 7395234, dtype: float64

In [18]:
accre[accre['END'].notnull()]['END'].dt.date

6640577    2020-10-01
6640635    2020-10-01
6640627    2020-10-01
6640651    2020-10-01
6640654    2020-10-01
              ...    
927386     2021-10-07
927387     2021-10-07
927388     2021-10-07
1491977    2021-10-07
1491978    2021-10-07
Name: END, Length: 7395234, dtype: object